# 📝 HOMEWORK 3 — GitHub Actions CI Pipeline for Data Quality

This notebook demonstrates how to create a **CI/CD pipeline** that automatically validates data quality on every commit using GitHub Actions.

## 🎯 Learning Objectives
- Set up a GitHub repository with data quality checks
- Create GitHub Actions workflow files
- Integrate Great Expectations / Pydantic validation in CI
- Handle validation failures with proper exit codes
- (Optional) Slack notifications on CI failure

## 📁 Repository Structure

```
your-repo/
├── .github/
│   └── workflows/
│       └── data_quality.yml      # GitHub Actions workflow
├── data/
│   └── amazon_sales.csv          # Dataset
├── src/
│   └── validate_data.py          # Validation script
├── requirements.txt              # Python dependencies
└── README.md
```

---
## 1️⃣ Prerequisites

Before starting:
- GitHub account
- Git installed locally
- Basic understanding of Git commands
- (Optional) Slack webhook URL for notifications

---
## 2️⃣ Create Repository Files

Let's create all the files needed for our CI pipeline.

### 2.1 requirements.txt

Python dependencies for the validation script.

In [ ]:
requirements_txt = """
great_expectations>=1.0.0
pydantic>=2.0.0
pandas>=2.0.0
requests>=2.28.0
""".strip()

# Save to file
with open('requirements.txt', 'w') as f:
    f.write(requirements_txt)

print("✅ requirements.txt created")
print(requirements_txt)

### 2.2 Validation Script (validate_data.py)

This script will be executed by GitHub Actions. It must:
- Load and validate the data
- Print a summary
- **Exit with code 1 if validation fails** (critical for CI!)

In [ ]:
validation_script = '''
"""
Data Quality Validation Script for CI/CD Pipeline
=================================================

This script validates the Amazon Sales dataset using Great Expectations.
It exits with code 1 if validation fails, causing the CI pipeline to fail.

Usage:
    python src/validate_data.py

Exit Codes:
    0 - All validations passed
    1 - One or more validations failed
"""

import sys
import os
import great_expectations as gx
import pandas as pd
import requests
import json
from datetime import datetime


# =============================================================================
# CONFIGURATION
# =============================================================================

DATA_PATH = "data/amazon_sales.csv"

# Slack webhook from environment variable (set in GitHub Secrets)
SLACK_WEBHOOK_URL = os.environ.get("SLACK_WEBHOOK_URL", "")

# Expected values
VALID_STATUSES = [
    "Cancelled", "Shipped", "Shipped - Delivered to Buyer", "Pending",
    "Shipped - Returned to Seller", "Shipped - Rejected by Buyer",
    "Shipped - Returning to Seller", "Shipped - Out for Delivery",
    "Shipped - Picked Up"
]
VALID_FULFILMENT = ["Merchant", "Amazon"]
VALID_CURRENCIES = ["INR"]
VALID_COUNTRIES = ["IN"]


# =============================================================================
# VALIDATION FUNCTIONS
# =============================================================================

def load_data(path: str) -> pd.DataFrame:
    """Load CSV data into DataFrame."""
    print(f"\n📂 Loading data from: {path}")
    
    if not os.path.exists(path):
        print(f"❌ ERROR: File not found: {path}")
        sys.exit(1)
    
    df = pd.read_csv(path)
    print(f"   Rows: {len(df)}, Columns: {len(df.columns)}")
    return df


def run_validation(df: pd.DataFrame) -> dict:
    """Run Great Expectations validation on the DataFrame."""
    print("\n🔍 Running Great Expectations Validation...")
    
    # Create ephemeral context
    context = gx.get_context()
    
    # Set up data source
    data_source = context.data_sources.add_pandas(name="pandas_source")
    data_asset = data_source.add_dataframe_asset(name="amazon_sales")
    batch_definition = data_asset.add_batch_definition_whole_dataframe("full_data")
    
    # Create expectation suite
    suite = gx.ExpectationSuite(name="amazon_sales_suite")
    
    # Add expectations
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToNotBeNull(column="Order ID")
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeUnique(column="Order ID")
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeBetween(column="Qty", min_value=0)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeBetween(column="Amount", min_value=0)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeInSet(column="Status", value_set=VALID_STATUSES)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeInSet(column="Fulfilment", value_set=VALID_FULFILMENT)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeInSet(column="currency", value_set=VALID_CURRENCIES)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToBeInSet(column="ship-country", value_set=VALID_COUNTRIES)
    )
    suite.add_expectation(
        gx.expectations.ExpectColumnValuesToMatchRegex(column="Date", regex=r"^\\d{2}-\\d{2}-\\d{2}$")
    )
    
    # Add suite to context
    suite = context.suites.add(suite)
    
    # Create validation definition
    validation_definition = gx.ValidationDefinition(
        name="amazon_sales_validation",
        data=batch_definition,
        suite=suite
    )
    validation_definition = context.validation_definitions.add(validation_definition)
    
    # Run validation
    results = validation_definition.run(batch_parameters={"dataframe": df})
    
    return results


def process_results(results) -> dict:
    """Process validation results and return summary."""
    results_dict = results.to_json_dict()
    expectation_results = results_dict.get("results", [])
    
    passed = []
    failed = []
    
    for exp_result in expectation_results:
        exp_config = exp_result.get("expectation_config", {})
        exp_type = exp_config.get("type", "Unknown")
        column = exp_config.get("kwargs", {}).get("column", "N/A")
        success = exp_result.get("success", False)
        
        info = {
            "expectation": exp_type,
            "column": column,
            "result": exp_result.get("result", {})
        }
        
        if success:
            passed.append(info)
        else:
            failed.append(info)
    
    return {
        "success": results_dict.get("success", False),
        "total": len(expectation_results),
        "passed": passed,
        "failed": failed
    }


def print_summary(summary: dict):
    """Print validation summary to console."""
    print("\n" + "=" * 60)
    print("   DATA QUALITY VALIDATION SUMMARY")
    print("=" * 60)
    
    status = "✅ PASSED" if summary["success"] else "❌ FAILED"
    print(f"\n   Overall Status: {status}")
    print(f"   Total Expectations: {summary['total']}")
    print(f"   Passed: {len(summary['passed'])}")
    print(f"   Failed: {len(summary['failed'])}")
    
    if summary["failed"]:
        print("\n" + "-" * 60)
        print("   FAILED EXPECTATIONS:")
        print("-" * 60)
        for f in summary["failed"]:
            print(f"   ❌ {f['expectation']} (Column: {f['column']})")
            result = f.get('result', {})
            if result.get('unexpected_count'):
                print(f"      Unexpected count: {result['unexpected_count']}")
            if result.get('partial_unexpected_list'):
                print(f"      Sample values: {result['partial_unexpected_list'][:3]}")
    
    print("\n" + "=" * 60)


def send_slack_notification(summary: dict) -> bool:
    """Send Slack notification if webhook URL is configured."""
    if not SLACK_WEBHOOK_URL:
        print("\n⚠️  Slack notification skipped (SLACK_WEBHOOK_URL not set)")
        return False
    
    print("\n📤 Sending Slack notification...")
    
    emoji = "✅" if summary["success"] else "❌"
    color = "#36a64f" if summary["success"] else "#dc3545"
    status_text = "PASSED" if summary["success"] else "FAILED"
    
    # Get GitHub context if available
    repo = os.environ.get("GITHUB_REPOSITORY", "local")
    run_id = os.environ.get("GITHUB_RUN_ID", "N/A")
    actor = os.environ.get("GITHUB_ACTOR", "local")
    
    failed_text = ""
    if summary["failed"]:
        failed_text = "\\n".join([
            f"• `{f['expectation']}` ({f['column']})"
            for f in summary["failed"]
        ])
    
    message = {
        "attachments": [{
            "color": color,
            "blocks": [
                {
                    "type": "header",
                    "text": {
                        "type": "plain_text",
                        "text": f"{emoji} CI Data Quality: {status_text}",
                        "emoji": True
                    }
                },
                {
                    "type": "section",
                    "fields": [
                        {"type": "mrkdwn", "text": f"*Repository:*\\n{repo}"},
                        {"type": "mrkdwn", "text": f"*Triggered by:*\\n{actor}"},
                        {"type": "mrkdwn", "text": f"*Passed:*\\n{len(summary['passed'])} ✓"},
                        {"type": "mrkdwn", "text": f"*Failed:*\\n{len(summary['failed'])} ✗"}
                    ]
                }
            ]
        }]
    }
    
    if failed_text:
        message["attachments"][0]["blocks"].append({
            "type": "section",
            "text": {"type": "mrkdwn", "text": f"*Failed Checks:*\\n{failed_text}"}
        })
    
    try:
        response = requests.post(
            SLACK_WEBHOOK_URL,
            data=json.dumps(message),
            headers={"Content-Type": "application/json"}
        )
        if response.status_code == 200:
            print("   ✅ Slack notification sent!")
            return True
        else:
            print(f"   ❌ Slack failed: {response.status_code}")
            return False
    except Exception as e:
        print(f"   ❌ Slack error: {e}")
        return False


# =============================================================================
# MAIN
# =============================================================================

def main():
    """Main entry point for CI pipeline."""
    print("\n" + "=" * 60)
    print("   🚀 DATA QUALITY CI PIPELINE")
    print("=" * 60)
    print(f"   Timestamp: {datetime.now().isoformat()}")
    
    # Load data
    df = load_data(DATA_PATH)
    
    # Run validation
    results = run_validation(df)
    
    # Process results
    summary = process_results(results)
    
    # Print summary
    print_summary(summary)
    
    # Send Slack notification
    send_slack_notification(summary)
    
    # Exit with appropriate code
    if not summary["success"]:
        print("\n💥 Validation FAILED - Exiting with code 1")
        sys.exit(1)
    else:
        print("\n🎉 Validation PASSED - Exiting with code 0")
        sys.exit(0)


if __name__ == "__main__":
    main()
'''.strip()

# Create src directory and save script
import os
os.makedirs('src', exist_ok=True)

with open('src/validate_data.py', 'w') as f:
    f.write(validation_script)

print("✅ src/validate_data.py created")
print(f"   Lines: {len(validation_script.splitlines())}")

### 2.3 GitHub Actions Workflow (data_quality.yml)

This is the heart of the CI pipeline. It defines:
- **Triggers**: When to run (push, pull_request, manual)
- **Environment**: Python version, dependencies
- **Steps**: What to execute

In [ ]:
workflow_yaml = '''
# ============================================================================
# Data Quality Validation Workflow
# ============================================================================
# This workflow runs data quality checks on every push and pull request.
# It will fail the build if any validation fails.
#
# Features:
#   - Runs on push to main/master
#   - Runs on pull requests
#   - Can be triggered manually
#   - Sends Slack notifications (optional)
# ============================================================================

name: Data Quality Validation

# ============================================================================
# TRIGGERS
# ============================================================================
on:
  # Run on push to main branches
  push:
    branches:
      - main
      - master
    paths:
      - 'data/**'           # Only run when data files change
      - 'src/**'            # Or when validation scripts change
      - '.github/workflows/**'
  
  # Run on pull requests
  pull_request:
    branches:
      - main
      - master
  
  # Allow manual trigger from GitHub UI
  workflow_dispatch:

# ============================================================================
# JOBS
# ============================================================================
jobs:
  validate:
    name: Validate Data Quality
    runs-on: ubuntu-latest
    
    steps:
      # ------------------------------------------
      # Step 1: Checkout repository
      # ------------------------------------------
      - name: 📥 Checkout code
        uses: actions/checkout@v4
      
      # ------------------------------------------
      # Step 2: Set up Python
      # ------------------------------------------
      - name: 🐍 Set up Python
        uses: actions/setup-python@v5
        with:
          python-version: '3.11'
          cache: 'pip'  # Cache pip dependencies
      
      # ------------------------------------------
      # Step 3: Install dependencies
      # ------------------------------------------
      - name: 📦 Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
      
      # ------------------------------------------
      # Step 4: Run validation
      # ------------------------------------------
      - name: 🔍 Run Data Quality Validation
        env:
          SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK_URL }}
        run: |
          python src/validate_data.py
      
      # ------------------------------------------
      # Step 5: Upload validation report (optional)
      # ------------------------------------------
      - name: 📊 Upload validation results
        if: always()  # Run even if validation fails
        uses: actions/upload-artifact@v4
        with:
          name: validation-results
          path: |
            valid_rows.csv
            invalid_rows.csv
          retention-days: 7
          if-no-files-found: ignore
'''.strip()

# Create .github/workflows directory and save workflow
os.makedirs('.github/workflows', exist_ok=True)

with open('.github/workflows/data_quality.yml', 'w') as f:
    f.write(workflow_yaml)

print("✅ .github/workflows/data_quality.yml created")
print(f"   Lines: {len(workflow_yaml.splitlines())}")

### 2.4 Sample Data

Copy the dataset to the data folder.

In [ ]:
# Create data directory
os.makedirs('data', exist_ok=True)

# Sample dataset with intentional errors
sample_data = """index,Order ID,Date,Status,Fulfilment,Sales Channel ,ship-service-level,Style,SKU,Category,Size,ASIN,Courier Status,Qty,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,S,B09KXVBD7Z,,0,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,,False,Easy Ship,
1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,3XL,B09K3WFS32,Shipped,1,INR,406.0,BENGALURU,KARNATAKA,560085.0,IN,"Amazon PLCC Free-Financing",False,Easy Ship,
2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,XL,B07WV4JV4D,Shipped,1,INR,329.0,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping,True,,
3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,L,B099NRCT7B,,0,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,,False,Easy Ship,
4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,3XL,B098714BZP,Shipped,1,INR,574.0,CHENNAI,TAMIL NADU,600073.0,IN,,False,,
5,404-1490984-4578765,04-30-22,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,XL,B08YN7XDSG,Shipped,1,INR,824.0,GHAZIABAD,UTTAR PRADESH,201102.0,IN,IN Core Free Shipping,False,,
6,408-5748499-6859555,04-30-22,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,L,B08CMHNWBN,Shipped,1,INR,653.0,CHANDIGARH,CHANDIGARH,160036.0,IN,IN Core Free Shipping,False,,
7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3405,JNE3405-KR-S,kurta,S,B081WX4G4Q,Shipped,1,INR,399.0,HYDERABAD,TELANGANA,500032.0,IN,"Amazon PLCC Free-Financing",False,Easy Ship,
8,407-5443024-5233168,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,3XL,B08L91ZZXN,Cancelled,0,INR,,HYDERABAD,TELANGANA,500008.0,IN,IN Core Free Shipping,False,,
9,402-4393761-0311520,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3461,JNE3461-KR-XXL,kurta,XXL,B08B3XF5MH,Shipped,1,INR,363.0,Chennai,TAMIL NADU,600041.0,IN,,False,,
10,407-5633625-6970741,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3160,JNE3160-KR-G-S,kurta,S,B07K3YQLF1,Shipped,1,INR,685.0,CHENNAI,TAMIL NADU,600073.0,IN,,False,,
11,,04-30-22,Pending,Merchant,Amazon.in,Standard,JNE1234,JNE1234-KR-M,kurta,M,B07K3YQLF2,Pending,2,INR,450.0,DELHI,DELHI,110001.0,IN,,False,Easy Ship,
12,408-1234567-8901234,04-30-22,Shipped,Amazon,Amazon.in,Expedited,SET100,SET100-KR-L,Set,L,B08L91ZZXN,Shipped,-1,INR,550.0,MUMBAI,MAHARASHTRA,400001.0,IN,,False,,
13,409-9876543-2109876,05-01-22,Shipped,Amazon,Amazon.in,Expedited,JNE5000,JNE5000-KR-XL,kurta,XL,B07WV4JV4E,Shipped,1,USD,299.0,PUNE,MAHARASHTRA,411001.0,US,,False,,
14,410-1111111-2222222,invalid-date,Shipped,Amazon,Amazon.in,Standard,TOP200,TOP200-TU-M,Top,M,B098714BZQ,Shipped,1,INR,399.0,JAIPUR,RAJASTHAN,302001.0,IN,,False,,
"""

with open('data/amazon_sales.csv', 'w') as f:
    f.write(sample_data)

print("✅ data/amazon_sales.csv created")
print(f"   Rows: 15 (including 6 intentional errors)")

### 2.5 README.md

In [ ]:
readme_content = '''
# Data Quality CI Pipeline

![Data Quality](https://github.com/YOUR_USERNAME/YOUR_REPO/actions/workflows/data_quality.yml/badge.svg)

This repository demonstrates automated data quality validation using GitHub Actions.

## 🚀 Features

- **Automated Validation**: Runs on every push and pull request
- **Great Expectations**: Industry-standard data validation framework
- **Slack Notifications**: Get alerted when validation fails
- **CI/CD Integration**: Fails the build if data quality issues are detected

## 📁 Structure

```
├── .github/workflows/
│   └── data_quality.yml    # GitHub Actions workflow
├── data/
│   └── amazon_sales.csv    # Sample dataset
├── src/
│   └── validate_data.py    # Validation script
├── requirements.txt        # Python dependencies
└── README.md
```

## 🔧 Setup

1. Fork this repository
2. (Optional) Add Slack webhook to repository secrets:
   - Go to Settings → Secrets and variables → Actions
   - Add `SLACK_WEBHOOK_URL`

## 🧪 Validation Rules

| Column | Rule | Description |
|--------|------|-------------|
| Order ID | Not Null | Every order must have an ID |
| Order ID | Unique | No duplicate order IDs |
| Qty | >= 0 | Quantity cannot be negative |
| Amount | >= 0 | Amount cannot be negative |
| Status | In Set | Must be valid status value |
| currency | = INR | Must be INR |
| ship-country | = IN | Must be IN |
| Date | Format | Must match MM-DD-YY |

## 🏃 Local Testing

```bash
# Install dependencies
pip install -r requirements.txt

# Run validation
python src/validate_data.py
```

## 📝 License

MIT License
'''.strip()

with open('README.md', 'w') as f:
    f.write(readme_content)

print("✅ README.md created")

---
## 3️⃣ View Created Files

In [ ]:
# List all created files
import os

print("📁 Repository Structure:")
print("=" * 40)

for root, dirs, files in os.walk('.'):
    # Skip hidden directories except .github
    dirs[:] = [d for d in dirs if not d.startswith('.') or d == '.github']
    
    level = root.replace('.', '').count(os.sep)
    indent = '  ' * level
    print(f'{indent}{os.path.basename(root)}/')
    
    subindent = '  ' * (level + 1)
    for file in files:
        if not file.startswith('.'):
            print(f'{subindent}{file}')

---
## 4️⃣ Test Validation Locally

Before pushing to GitHub, test the validation script locally.

In [ ]:
# Test the validation script (simulating what CI will do)
# Note: In a real scenario, you would run this from command line

print("🧪 Testing validation script locally...\n")
print("Run this command in your terminal:")
print("-" * 40)
print("python src/validate_data.py")
print("-" * 40)

---
## 5️⃣ Push to GitHub

Now push everything to GitHub to trigger the CI pipeline.

In [ ]:
# Git commands to push to GitHub
git_commands = """
# ============================================
# Git Commands to Push to GitHub
# ============================================

# 1. Initialize git repository (if not already)
git init

# 2. Add all files
git add .

# 3. Commit
git commit -m "Add data quality CI pipeline"

# 4. Create GitHub repository and add remote
# (Create repo on GitHub first, then:)
git remote add origin https://github.com/YOUR_USERNAME/YOUR_REPO.git

# 5. Push to GitHub
git branch -M main
git push -u origin main

# ============================================
# The CI pipeline will automatically run!
# Check: https://github.com/YOUR_USERNAME/YOUR_REPO/actions
# ============================================
"""

print(git_commands)

---
## 6️⃣ Configure Slack Notifications (Optional)

To receive Slack notifications when validation fails:

In [ ]:
slack_setup_instructions = """
# ============================================
# Setting Up Slack Notifications
# ============================================

1. Create Slack Webhook:
   - Go to https://api.slack.com/apps
   - Create New App → From scratch
   - Add "Incoming Webhooks" feature
   - Create webhook for your channel
   - Copy the webhook URL

2. Add to GitHub Secrets:
   - Go to your repository on GitHub
   - Settings → Secrets and variables → Actions
   - Click "New repository secret"
   - Name: SLACK_WEBHOOK_URL
   - Value: (paste your webhook URL)
   - Click "Add secret"

3. That's it! 
   The CI pipeline will now send Slack notifications
   whenever validation runs.

# ============================================
"""

print(slack_setup_instructions)

---
## 7️⃣ Understanding the CI Workflow

Let's break down what happens when you push code:

In [ ]:
workflow_explanation = """
┌─────────────────────────────────────────────────────────────┐
│                    CI/CD PIPELINE FLOW                      │
└─────────────────────────────────────────────────────────────┘

    ┌──────────┐
    │ git push │
    └────┬─────┘
         │
         ▼
    ┌──────────────────────┐
    │ GitHub detects push  │
    │ to main branch       │
    └──────────┬───────────┘
               │
               ▼
    ┌──────────────────────┐
    │ Workflow triggered   │
    │ (data_quality.yml)   │
    └──────────┬───────────┘
               │
               ▼
    ┌──────────────────────┐
    │ 1. Checkout code     │
    │ 2. Setup Python 3.11 │
    │ 3. Install deps      │
    │ 4. Run validation    │
    └──────────┬───────────┘
               │
         ┌─────┴─────┐
         │           │
         ▼           ▼
    ┌─────────┐ ┌─────────┐
    │ Exit 0  │ │ Exit 1  │
    │ SUCCESS │ │ FAILURE │
    └────┬────┘ └────┬────┘
         │           │
         ▼           ▼
    ┌─────────┐ ┌─────────────┐
    │ ✅ Pass │ │ ❌ Fail     │
    │ Green   │ │ Red         │
    │ badge   │ │ Block merge │
    └─────────┘ └─────────────┘
                      │
                      ▼
               ┌─────────────┐
               │ 📱 Slack    │
               │ notification│
               └─────────────┘
"""

print(workflow_explanation)

---
## 8️⃣ Branch Protection (Best Practice)

To prevent merging code that fails validation:

In [ ]:
branch_protection = """
# ============================================
# Setting Up Branch Protection Rules
# ============================================

1. Go to your repository on GitHub

2. Settings → Branches

3. Click "Add branch protection rule"

4. Configure:
   - Branch name pattern: main
   - ✅ Require a pull request before merging
   - ✅ Require status checks to pass before merging
   - ✅ Require branches to be up to date before merging
   - Select status check: "Validate Data Quality"

5. Click "Create"

Now:
- Direct pushes to main are blocked
- PRs must pass data quality checks
- Bad data can't be merged! 🛡️

# ============================================
"""

print(branch_protection)

---
## 📚 Summary

In this homework, we learned:

1. **GitHub Actions Basics**: Workflows, triggers, jobs, steps
2. **CI/CD for Data**: Automated validation on every commit
3. **Exit Codes**: How to signal success/failure to CI
4. **Secrets Management**: Storing Slack webhook securely
5. **Branch Protection**: Preventing bad data from being merged

### 🏋️ Bonus Exercises

1. Add a **scheduled run** (e.g., every day at 8 AM)
2. Create a **Pydantic validation** workflow
3. Add **data profiling** step that uploads a report
4. Implement **multi-environment** validation (dev, staging, prod)

### 🔗 Resources

- [GitHub Actions Documentation](https://docs.github.com/en/actions)
- [GitHub Actions Workflow Syntax](https://docs.github.com/en/actions/reference/workflow-syntax-for-github-actions)
- [Great Expectations + GitHub Actions](https://docs.greatexpectations.io/docs/deployment_patterns/how_to_use_great_expectations_with_github_actions/)
- [Slack Incoming Webhooks](https://api.slack.com/messaging/webhooks)